In [ ]:
import {PromptTemplate} from '@langchain/core/prompts';

const greetingPrompt = new PromptTemplate({
    inputVariables:[],
    template:'hello world'
})

const formattedGreetingPrompt = await greetingPrompt.format()

console.log(formattedGreetingPrompt);


In [ ]:
const personalizedGreetingPrompt=new PromptTemplate({
    inputVariables:['name'],
    template:'hello,{name}'
})

const formattedPersonalizedGreeting = await personalizedGreetingPrompt.format({name:'kai'})

console.log(formattedPersonalizedGreeting);


In [ ]:
const multiVariableGreetingPrompt = new PromptTemplate({
  inputVariables: ["timeOfDay", "name"],
  template: "good {timeOfDay},{name}",
});

const formattedmultiVariableGreeting = await multiVariableGreetingPrompt.format(
  {
    timeOfDay: "morning",
    name: "kai",
  }
);

console.log(formattedmultiVariableGreeting);


In [ ]:
const multiVariableGreetingPrompt = new PromptTemplate({
  inputVariables: ["timeOfDay", "name"],
  template: "good {timeOfDay},{{name}}",
});

const formattedmultiVariableGreeting = await multiVariableGreetingPrompt.format(
  {
    timeOfDay: "morning",
    name: "kai",
  }
);

console.log(formattedmultiVariableGreeting);


In [ ]:
const autoInferTemplate = PromptTemplate.fromTemplate(
  "good {timeOfDay},{name}"
);
// console.log(autoInferTemplate.inputVariables);

const formattedAutoInferTemplate = await autoInferTemplate.format({
  timeOfDay: "morning",
  name: "kai",
});

console.log(formattedAutoInferTemplate);


In [ ]:
const initialPrompt = new PromptTemplate({
    template:'这是一个{type}，它是{item}',
    inputVariables:['type','item']
})

const partialedPrompt=await initialPrompt.partial({
    type:'工具'
})

const formattedPrompt = await partialedPrompt.format({
  item: "锤子",
});

console.log(formattedPrompt);

const formattedPrompt2 = await partialedPrompt.format({
  item: "锥子",
});

console.log(formattedPrompt2);


In [ ]:
const getCurrentDateStr = () => {
  return new Date().toLocaleDateString();
};

const promptWithDate = new PromptTemplate({
  template: "今天是{date},{activity}",
  inputVariables: ["date", "activity"],
});

const partialedPromptWithDate = await promptWithDate.partial({
  date: getCurrentDateStr,
});

const formattedPromptWithDate = await partialedPromptWithDate.format({
  activity: "我们去爬山",
});

console.log(formattedPromptWithDate);


In [ ]:
const getCurrentDateStr = () => {
  return new Date().toLocaleDateString();
};

function generateGreeting(timeOfDay) {
  return () => {
    const date = getCurrentDateStr();
    switch (timeOfDay) {
      case "morining":
        return date + " 早上好";
      case "afternoon":
        return date + " 下午好";
      case "evening":
        return date + " 晚上好";
      default:
        return date + " 你好";
    }
  };
}

const prompt = new PromptTemplate({
  template: "{greeting}!",
  inputVariables: ["greeting"],
});

const currentTimeOfDay = "afternoon";
const partialPrompt = await prompt.partial({
  greeting: generateGreeting(currentTimeOfDay),
});

const formattedPrompt =await partialPrompt.format();

console.log(formattedPrompt);


In [ ]:
import { SystemMessagePromptTemplate } from "@langchain/core/prompts";

const translateInstructionTemplate = SystemMessagePromptTemplate.fromTemplate(
  "你是一个专业的翻译员，你的任务是将文本从{source_lang}翻译成{target_lang}。"
);


In [ ]:
import { HumanMessagePromptTemplate } from "@langchain/core/prompts";

const userQuestionTemplate = HumanMessagePromptTemplate.fromTemplate('请翻译这句话：{text}')

In [ ]:
import { ChatPromptTemplate } from "@langchain/core/prompts";

const chatPrompt = ChatPromptTemplate.fromMessages([
  translateInstructionTemplate,
  userQuestionTemplate,
]);


In [ ]:
const formattedChatPrompt =await chatPrompt.format({
  source_lang: "中文",
  target_lang: "法语",
  text:'你好，世界',
});

console.log(formattedChatPrompt);


In [ ]:
const systemTemplate =
  "你是一个专业的翻译员，你的任务是将文本从{source_lang}翻译成{target_lang}。";
  const humanTemplate = "请翻译这句话：{text}";
  const chatPrompt = ChatPromptTemplate.fromMessages([
    ['system',systemTemplate],
    ['human',humanTemplate]
  ])

  const formattedChatPrompt = await chatPrompt.format({
    source_lang: "中文",
    target_lang: "法语",
    text: "你好，世界",
  });

  console.log(formattedChatPrompt);

In [ ]:
import { load } from "dotenv";
import { ChatMoonshot } from "@langchain/community/chat_models/moonshot";
import { StringOutputParser } from "@langchain/core/output_parsers";
import { ChatPromptTemplate } from "@langchain/core/prompts";

const env = await load();

const process = {
  env,
};

const chatModel = new ChatMoonshot({
  apiKey: process.env.OPENAI_API_KEY,
});

const outputParser = new StringOutputParser();

const systemTemplate =
  "你是一个专业的翻译员，你的任务是将文本从{source_lang}翻译成{target_lang}。";
const humanTemplate = "请翻译这句话：{text}";
const chatPrompt = ChatPromptTemplate.fromMessages([
  ["system", systemTemplate],
  ["human", humanTemplate],
]);

const chain = chatPrompt.pipe(chatModel).pipe(outputParser);

await chain.invoke({
  source_lang: "中文",
  target_lang: "法语",
  text: "你好，世界",
});


In [ ]:
import {
  PromptTemplate,
  PipelinePromptTemplate,
} from "@langchain/core/prompts";

const getCurrentDateStr = () => {
  return new Date().toLocaleDateString();
};

const fullPrompt = PromptTemplate.fromTemplate(`
    你是一个只能饿管家，今天是{date}，你的主人信息是{info}，
    根据上下文，完成主人的需求
    {task}`);

const datePrompt = PromptTemplate.fromTemplate("{date}，现在是{period}");
const periodPrompt = await datePrompt.partial({
  date: getCurrentDateStr,
});

const infoPrompt = PromptTemplate.fromTemplate("姓名是{name}，性别是{gender}");

const taskPrompt = PromptTemplate.fromTemplate(`
    我想吃{period}的{food}。
    在重复一遍我的信息{info}`);

const composedPrompt = new PipelinePromptTemplate({
  pipelinePrompts: [
    { name: "date", prompt: periodPrompt },
    { name: "info", prompt: infoPrompt },
    { name: "task", prompt: taskPrompt },
  ],
  finalPrompt: fullPrompt,
});

const formattedPrompt = await composedPrompt.format({
  period: "早上",
  name: "张三",
  gender: "male",
  food: "lemon",
});
console.log(formattedPrompt);



    你是一个只能饿管家，今天是7/23/2025，现在是早上，你的主人信息是姓名是张三，性别是male，
    根据上下文，完成主人的需求
    
    我想吃早上的lemon。
    在重复一遍我的信息姓名是张三，性别是male
